In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用Vertex AI进行预测性维护

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/predictive_maintainance/predictive_maintenance_usecase.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
    <td>
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/predictive_maintainance/predictive_maintenance_usecase.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png\" alt="Colab logo"> 在Colab中运行
        </a>
  </td>
  <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/workbench/predictive_maintainance/predictive_maintenance_usecase.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>


## 目录
* [概述](#section-1)
* [目标](#section-2)
* [数据集](#section-3)
* [成本](#section-4)
* [数据分析](#section-5)
* [拟合回归模型](#section-6)
* [评估训练模型](#section-7)
* [保存模型](#section-8)
* [使用执行器端到端运行笔记本](#section-9)
* [在Vertex AI上托管模型](#section-10)
  * [创建端点](#section-11)
  * [将模型部署到创建的端点](#section-12)
  * [测试调用端点](#section-13)
* [清理](#section-14)

## 概述
<a name="section-1"></a>

在这个笔记本中，您将使用机器学习技术，在工业数据上进行预测性维护用例，将机器学习模型部署到Vertex AI，并使用Vertex AI Workbench的executor功能自动化工作流程。

*注意：此笔记本文件是为在[Vertex AI Workbench托管的笔记本](https://console.cloud.google.com/vertex-ai/workbench/list/managed)实例中使用XGBoost（Local）内核而开发的。此笔记本的一些组件在其他笔记本环境中可能无法正常运行。*

了解有关[Vertex AI Workbench](https://cloud.google.com/vertex-ai/docs/workbench/introduction)和[Vertex AI 训练](https://cloud.google.com/vertex-ai/docs/training/custom-training)的更多信息。

### 目标
<a name="section-2"></a>

在本教程中，您将学习如何使用Vertex AI Workbench中的执行器功能来自动化一个工作流程，用于训练和部署模型。

本教程使用以下谷歌云ML服务：

- `Vertex AI 训练`
- `Vertex AI 模型评估`

执行的步骤包括：

- 从云存储桶中加载所需数据集。
- 分析数据集中的字段。
- 选择用于预测性维护模型的必要数据。
- 训练用于预测剩余有效寿命的XGBoost回归模型。
- 评估模型。
- 使用Executor将笔记本端到端作为训练作业运行。
- 在Vertex AI上部署模型。
- 清理。

### 数据集
<a name="section-3"></a>

本笔记本中使用的数据集是[美国国家航空航天局涡轮风扇发动机退化模拟数据集](https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository/)的一部分，其中包含了四组舰队发动机在不同操作条件和故障模式下的模拟时间序列数据。本笔记本中使用了保存在公共云存储桶中的此数据集的一个版本。在本笔记本中，使用其中一个发动机的模拟数据（FD001）来分析并训练一个可以预测发动机剩余寿命的模型。

### 费用
<a name="section-4"></a>

本教程使用谷歌云的以下计费组件：

- Vertex AI
- 云存储

了解有关 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [云存储 价格](https://cloud.google.com/storage/pricing)，并使用 [价格计算器](https://cloud.google.com/products/calculator/) 根据您的预期用量生成一个费用估算。

### 选择核心
在 Vertex AI 工作台的托管实例上运行此笔记本时，请选择 <b>XGBoost</b> 核心。否则，请确保以下库已在运行此笔记本的环境中安装。
- XGBoost
- Pandas
- Seaborn
- Sklearn

除上述库外，此笔记本还使用以下 google-cloud 库。

- google.cloud.aiplatform
- google.cloud.storage

## 安装

安装以下包以在 Vertex AI Workbench 管理的实例之外运行此笔记本。

In [ ]:
! pip3 install --quiet --upgrade   google-cloud-aiplatform \
                                    google-cloud-storage \
                                    xgboost \
                                    seaborn \
                                    scikit-learn \
                                    fsspec \
                                    gcsfs \
                                    pandas -q

### 仅限 Colab 使用：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### 开始之前

#### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
- 运行 `gcloud config list`
- 运行 `gcloud projects list`
- 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# set the project id
! gcloud config set project $PROJECT_ID

区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。
了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

### 验证您的Google Cloud帐户

根据您的Jupyter环境，您可能需要手动验证。请按照以下相关说明操作。

**1. Vertex AI Workbench**
- 无需操作，因为您已经验证过了。

**2. 本地JupyterLab实例，** 取消注释并运行。

In [ ]:
# ! gcloud auth login

3. 在 Colab 中取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账号或其他
- 在这里查看所有的认证选项：[Google Cloud Platform Jupyter Notebook认证指南](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/notebook_authentication_guide.ipynb)

### 创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶不存在时才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} {BUCKET_URI}

UUID

如果您正在进行实时教程会话，您可能正在使用共享的测试帐户或项目。为了避免用户在创建的资源上发生名称冲突，您可以为每个实例会话创建一个UUID，并将其附加到您在本教程中创建的资源的名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

导入所需的库###

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline
import os

import numpy as np
import seaborn as sns
import xgboost as xgb
from google.cloud import aiplatform, storage
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

加载数据并检查数据形状。

In [ ]:
# load the data from the source
INPUT_PATH = "gs://cloud-samples-data/ai-platform-unified/datasets/tabular/predictive_maintenance.csv"  # data source
raw_data = pd.read_csv(INPUT_PATH, sep=" ", header=None)
# check the data
print(raw_data.shape)
raw_data.head()

数据本身不包含任何特征名称，因此需要将其列命名更改。数据源已经提供了一些数据描述。显然，<b>ID</b>列代表机群引擎的单位编号，<b>Cycle</b>代表循环中的时间。 <b>OpSet1</b>，<b>Opset2</b>和<b>Opset3</b>代表原始数据源中描述的三个操作设置，并且对引擎性能有重大影响。其余字段显示从21个不同传感器收集的传感器读数。

In [ ]:
# name the columns (based on the original data source page)
raw_data = raw_data[[f for f in range(0, 26)]]
raw_data.columns = [
    "ID",
    "Cycle",
    "OpSet1",
    "OpSet2",
    "OpSet3",
    "SensorMeasure1",
    "SensorMeasure2",
    "SensorMeasure3",
    "SensorMeasure4",
    "SensorMeasure5",
    "SensorMeasure6",
    "SensorMeasure7",
    "SensorMeasure8",
    "SensorMeasure9",
    "SensorMeasure10",
    "SensorMeasure11",
    "SensorMeasure12",
    "SensorMeasure13",
    "SensorMeasure14",
    "SensorMeasure15",
    "SensorMeasure16",
    "SensorMeasure17",
    "SensorMeasure18",
    "SensorMeasure19",
    "SensorMeasure20",
    "SensorMeasure21",
]
raw_data.head()

## 数据分析
<a name="section-5"></a>
当前数据集包含各个单元 ID 的时间序列数据。数据以循环方式表示。让我们首先看看各个单元的循环数量分布。

In [ ]:
# plot the cycle count for each IDs
raw_data[["ID", "Cycle"]].groupby(by=["ID"]).count().plot(kind="bar", figsize=(12, 5))

平均而言，数据集中每个ID大约有225个周期。接下来，让我们检查字段的数据类型和数据中空记录的数量。

In [ ]:
# check the data-types
raw_data.info()

数据中没有空记录或任何分类字段。接下来，让我们检查字段的数值分布。

In [ ]:
# check the numerical characteristics of the data
raw_data.describe().T

特征**OpSet3**，**SensorMeasure1**，**SensorMeasure10**，**SensorMeasure18**和**SensorMeasure19**似乎在整个数据集中是常数，因此可以被消除。除了数据集中常数的字段外，高度相关的字段也可以考虑删除。数据中存在高度相关的字段通常会导致多重共线性情况，即使不会对准确性产生影响，也会不必要地增加特征空间的大小。这种字段可以通过相关矩阵和热力图来识别。

In [ ]:
# plot the correlation matrix
plt.figure(figsize=(15, 10))
cols = [
    i
    for i in raw_data.columns
    if i
    not in [
        "ID",
        "Cycle",
        "OpSet3",
        "SensorMeasure1",
        "SensorMeasure10",
        "SensorMeasure18",
        "SensorMeasure19",
    ]
]
corr_mat = raw_data[cols].corr()
matrix = np.triu(corr_mat)

sns.heatmap(corr_mat, annot=True, mask=matrix, fmt=".1g")
plt.show()

字段**SensorMeasure7**、**SensorMeasure12**、**SensorMeasure20**和**SensorMeasure21**与许多其他字段高度相关。这些字段可以省略。此外，**SensorMeasure8**、**SensorMeasure11**和**SensorMeasure4**似乎彼此高度相关，因此可以保留其中任何一个，例如**SensorMeasure4**，其余可以省略。

In [ ]:
cols = [
    i
    for i in cols
    if i
    not in [
        "SensorMeasure7",
        "SensorMeasure12",
        "SensorMeasure20",
        "SensorMeasure21",
        "SensorMeasure8",
        "SensorMeasure11",
    ]
]
corr_mat = raw_data[cols].corr()
matrix = np.triu(corr_mat)
plt.figure(figsize=(9, 5))
sns.heatmap(corr_mat, annot=True, mask=matrix, fmt=".1g")
plt.show()

由于当前的目标是预测每个单位（ID）的剩余可用寿命（RUL），需要确定目标变量。由于您正在处理表示单位寿命的时间序列数据，单位的剩余可用寿命可以通过将当前周期从该单位的最大周期中减去来计算。

RUL = 最大周期 - 当前周期
## RUL 的计算和特征选择

In [ ]:
# get max-cycle of the ids
cols = ["ID", "Cycle"] + cols
max_cycles_df = (
    raw_data.groupby(["ID"], sort=False)["Cycle"]
    .max()
    .reset_index()
    .rename(columns={"Cycle": "MaxCycleID"})
)
# merge back to original dataset
FD001_df = pd.merge(raw_data, max_cycles_df, how="inner", on="ID")
# calculate rul from max-cycle and current-cycle
FD001_df["RUL"] = FD001_df["MaxCycleID"] - FD001_df["Cycle"]

为了确保目标字段的正确生成，可以绘制RUL字段。

In [ ]:
# plot the RUL vs Cycles
one_engine = []
for i, r in FD001_df.iterrows():
    rul = r["RUL"]
    one_engine.append(rul)
    if rul == 0:
        plt.plot(one_engine)
        one_engine = []

plt.grid()

以上图表表明，寿命剩余（RUL），也就是剩余周期，随着当前周期的增加而减少，这是可以预期的。此外，让我们看看当前数据集中的其他字段与RUL的关系如何。

In [ ]:
# plot feature vs the RUL
def plot_feature(feature):
    plt.figure(figsize=(10, 5))
    for i in FD001_df["ID"].unique():
        if i % 10 == 0:  # only plot every 10th ID
            plt.plot("RUL", feature, data=FD001_df[FD001_df["ID"] == i])
    plt.xlim(250, 0)  # reverse the x-axis so RUL counts down to zero
    plt.xticks(np.arange(0, 275, 25))
    plt.ylabel(feature)
    plt.xlabel("RUL")
    plt.show()


for i in cols:
    if i not in ["ID", "Cycle"]:
        plot_feature(i)

可以从上面细胞的结果中得出以下一组观察：
- **传感器测量5**和**传感器测量16**的字段与剩余寿命（RUL）没有太大变化，看起来一直保持恒定。因此，它们可以被移除。
- **传感器测量2**、**传感器测量3**、**传感器测量4**、**传感器测量13**、**传感器测量15**和**传感器测量17**显示出类似的上升趋势。
- **传感器测量9**和**传感器测量14**显示出类似的趋势。
- **传感器测量6**大部分时间显示为一条水平线，只有在很少的地方有变化，因此可以被忽略。

In [ ]:
# remove the unnecessary fields
cols = [
    i
    for i in cols
    if i not in ["ID", "SensorMeasure5", "SensorMeasure6", "SensorMeasure16"]
]
cols

##将数据分为训练集和测试集

将选定特征的数据集分为训练集和测试集。

In [ ]:
# split data into train and test
X = FD001_df[cols].copy()
y = FD001_df["RUL"].copy()

# split the data into 70-30 ratio of train-test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.7, random_state=36
)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

## 拟合回归模型
<a name="section-6"></a>

使用XGBoost库初始化并训练一个回归模型，以计算得到的RUL作为目标特征。

In [ ]:
model = xgb.XGBRegressor()
model.fit(X_train, y_train)

## 评估训练好的模型
<a name="section-7"></a>

检查模型在训练集和测试集上的R2分数。

In [ ]:
# print test R2 score
y_train_pred = model.predict(X_train)
train_score = r2_score(y_train, y_train_pred)
y_test_pred = model.predict(X_test)
test_score = r2_score(y_test, y_test_pred)
print("Train score:", train_score)
print("Test score:", test_score)

检查训练集和测试集上的 RMSE 错误。

In [ ]:
# print train and test RMSEs
train_error = mean_squared_error(y_train, y_train_pred, squared=False)
test_error = mean_squared_error(y_test, y_test_pred, squared=False)
print("Train error:", train_error)
print("Test error:", test_error)

将预测值绘制成目标值的图表。图表越接近通过原点且具有单位斜率的直线，模型越好。

In [ ]:
# plot the train and test predictions
plt.scatter(y_train, y_train_pred)
plt.xlabel("Target")
plt.ylabel("Prediction")
plt.title("Train")
plt.show()
plt.scatter(y_test, y_test_pred)
plt.xlabel("Target")
plt.ylabel("Prediction")
plt.title("Test")
plt.show()

保存模型
<a name="section-8"></a>

将模型保存到一个booster文件中。

In [ ]:
# save the trained model to a local file "model.bst"
FILE_NAME = "model.bst"
model.save_model(FILE_NAME)

将模型复制到云存储桶

In [ ]:
# Upload the saved model file to Cloud Storage
BLOB_PATH = "mfg_predictive_maintenance/"
BLOB_NAME = os.path.join(BLOB_PATH, FILE_NAME)
bucket = storage.Client().bucket(BUCKET_URI[5:])
blob = bucket.blob(BLOB_NAME)
blob.upload_from_filename(FILE_NAME)

## 使用执行器端到端运行笔记本
<a name="section-9"></a>

**注意：**在 Vertex AI Workbench 上的托管实例上运行此笔记本时才考虑此部分。
### 自动化笔记本执行
到目前为止遵循的所有步骤都可以作为一个训练作业运行，而无需使用任何额外的代码，使用 Vertex AI Workbench 执行器。执行器可以帮助您选择环境、机器类型、输入参数和其他特征，在设置执行后，笔记本将作为 Vertex AI 自定义训练中的作业执行。您可以在左侧边栏的 Executor 面板中监视您的作业。

<img src="images/executor.PNG">

执行器还可以让您在自动化运行时选择环境和机器类型，类似于 Vertex AI 训练作业，无需切换到训练作业用户界面。除了默认复制现有内核的自定义容器外，还可以选择预构建的环境，如 TensorFlow Enterprise、PyTorch 等来运行笔记本。可以通过从可用机器类型列表中选择来指定所需的计算能力，包括 GPU。

### 在执行器上安排运行

笔记本运行也可以使用执行器定期安排运行。为此，选择以基于计划的重复执行为运行类型，而不是一次性执行。创建执行时可以提供作业的频率和执行时间。

<img src="https://storage.googleapis.com/gweb-cloudblog-publish/images/7_Vertex_AI_Workbench.max-1100x1100.jpg">

### 参数化变量

执行器可以让您使用不同的输入参数集运行笔记本。如果在任何笔记本单元格中添加了参数标记，您可以将参数值传递给执行器。有关如何使用此功能的更多信息可以在[此博客](https://cloud.google.com/blog/products/ai-machine-learning/schedule-and-execute-notebooks-with-vertex-ai-workbench)中找到。 

<img src="https://storage.googleapis.com/gweb-cloudblog-publish/images/6_Vertex_AI_Workbench.max-700x700.jpg">

## 在 Vertex AI 上托管模型
<a name="section-10"></a>

### 创建一个模型资源

可以使用 Vertex AI SDK 轻松部署在 Cloud Storage 中保存的模型。为此，首先创建一个模型资源。

In [ ]:
ARTIFACT_GCS_PATH = f"{BUCKET_URI}/{BLOB_PATH}"

为Vertex AI模型资源指定一个显示名称。

In [ ]:
# Set the model-dsiplay-name
MODEL_DISPLAY_NAME = "[your-model-display-name]"  # @param {type:"string"}

# Otherwise, use the default name
if (
    MODEL_DISPLAY_NAME == "[your-model-display-name]"
    or MODEL_DISPLAY_NAME is None
    or MODEL_DISPLAY_NAME == ""
):
    MODEL_DISPLAY_NAME = "pred_maint_model_" + UUID

print(MODEL_DISPLAY_NAME)

In [ ]:
# Create a Vertex AI model resource
aiplatform.init(project=PROJECT_ID, location=REGION)

model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=ARTIFACT_GCS_PATH,
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/xgboost-cpu.1-1:latest",
)

model.wait()

print(model.display_name)
print(model.resource_name)

创建一个端点
<a name="section-11"></a>

接下来，创建一个端点资源用于部署模型。

In [ ]:
# Set the endpoint-dsiplay-name
ENDPOINT_DISPLAY_NAME = "[your-endpoint-display-name]"  # @param {type:"string"}

# Otherwise, use the default name
if (
    ENDPOINT_DISPLAY_NAME == "[your-endpoint-display-name]"
    or ENDPOINT_DISPLAY_NAME is None
    or ENDPOINT_DISPLAY_NAME == ""
):
    ENDPOINT_DISPLAY_NAME = "pred_maint_endpoint_" + UUID

print(ENDPOINT_DISPLAY_NAME)

In [ ]:
# Create the Endpoint resource
endpoint = aiplatform.Endpoint.create(display_name=ENDPOINT_DISPLAY_NAME)

print(endpoint.display_name)
print(endpoint.resource_name)

### 部署模型到创建的终结点
<a name="section-12"></a>

配置以下参数并将模型部署到创建的终结点。

- `endpoint`: 使用 Vertex AI SDK 创建的 `Endpoint` 对象。
- `deployed_model_display_name`: 部署的显示名称。
- `machine_type`: 部署环境所需的机器类型。请参阅[这里](https://cloud.google.com/vertex-ai/docs/predictions/configure-compute)了解更多信息。

In [ ]:
# deploy the model to the endpoint
model.deploy(
    endpoint=endpoint,
    deployed_model_display_name=MODEL_DISPLAY_NAME + "_deployment",
    machine_type="n1-standard-2",
)

model.wait()

print(model.display_name)
print(model.resource_name)

## 调用端点进行测试
<a name="section-13"></a>

向部署的模型端点发送一些样本数据以获得预测结果。

In [ ]:
# get predictions on sample data
instances = X_test.iloc[0:2].to_numpy().tolist()
print(endpoint.predict(instances=instances).predictions)

清理
<a name="section-14"></a>

要清理本项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：
* Vertex AI模型
* Vertex AI端点
* Cloud Storage存储桶

将`delete_bucket`设置为**True**以删除Cloud Storage存储桶。

In [ ]:
# Undeploy all the models from the endpoint
endpoint.undeploy_all()

# Delete the endpoint resource
endpoint.delete()

# Delete the model resource
model.delete()

# Delete the Cloud Storage bucket
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI